**Develop a sentiment analysis model to predict the sentiment of movie reviews as positive or negative.**

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
imdb_reviews=pd.read_csv("imdb_reviews.csv")  #importing Training dataset
test_reviews=pd.read_csv("test_reviews.csv")  #importing test dataset

In [ ]:
imdb_reviews.head()  #checking if the train csv is getting loaded correctly

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


In [ ]:
test_reviews.head()  #checking if the test csv is getting uploaded correctly

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


Processing the Data

We need to convert the String data to numerical data to pass through our model. Using the word_indexes.csv file to map words to integers.

In [ ]:
word_index=pd.read_csv("word_indexes.csv") #checking if the word_indexed csv is getting uploaded correctly

In [ ]:
word_index.head()

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411


In [ ]:
word_index=dict(zip(word_index.Words,word_index.Indexes)) 

In [ ]:
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

Now we define a function review_encoder that encodes the reviews into integer format according to the mapping specified by word_index file.

In [ ]:
def review_encoder(text):
  arr=[word_index[word] for word in text]
  return arr

We split the reviews from their corresponding sentiments so that we can preprocess the reviews and sentiments separately and then later pass it to our model.


In [ ]:
train_data,train_labels=imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data, test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

Before transforming the reviews as integers we need to tokenize or split the review on the basis of whitespaces For eg.the string "The movie was wonderful" becomes ["The" , "movie" , "was" , "wonderful" ].

In [ ]:
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())

Since we have tokenized the reviews now we can apply the review_encoder function to each review and transform the reviews into integer format.

In [ ]:
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

After transforming, our reviews are going to look like this.

In [ ]:
train_data.head()

0    [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...
1    [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...
2    [1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...
3    [1, 4, 2, 2, 33, 2804, 4, 2040, 432, 111, 153,...
4    [1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...
Name: Reviews, dtype: object

We also need to encode the sentiments and we are labeling the positive sentiment as 1 and negative sentiment as 0.

In [ ]:
def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)

Before giving the review as an input to the model we need to perform following preprocessing steps:

The length of each review should be made equal for the model to be working correctly.

We have chosen the length of each review to be 500.

If the review is longer than 500 words we are going to cut the extra part of the review.

If the review is contains less than 500 words we are going to pad the review with zeros to increase its length to 500.

In [ ]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

**Model**

Our model is a neural network and it consits of the following layers :

one word embedding layer which creates word embeddings of length 16 from integer encoded review.

second layer is global average pooling layer which is used to prevent overfitting by reducing the number of parameters.

then a dense layer which has 16 hidden units and uses relu as activation function

the final layer is the output layer which uses sigmoid as activation function

In [ ]:
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In the next step we are going to train the model on our downloaded IMDB dataset.

In [ ]:
#training the model
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 3s 48ms/step - loss: 0.6919 - accuracy: 0.5584 - val_loss: 0.6900 - val_accuracy: 0.6175
Epoch 2/30
49/49 [==============================] - 2s 40ms/step - loss: 0.6854 - accuracy: 0.7201 - val_loss: 0.6798 - val_accuracy: 0.7128
Epoch 3/30
49/49 [==============================] - 2s 45ms/step - loss: 0.6672 - accuracy: 0.7591 - val_loss: 0.6553 - val_accuracy: 0.7519
Epoch 4/30
49/49 [==============================] - 3s 65ms/step - loss: 0.6312 - accuracy: 0.7765 - val_loss: 0.6134 - val_accuracy: 0.7848
Epoch 5/30
49/49 [==============================] - 2s 45ms/step - loss: 0.5790 - accuracy: 0.8077 - val_loss: 0.5618 - val_accuracy: 0.8024
Epoch 6/30
49/49 [==============================] - 2s 40ms/step - loss: 0.5200 - accuracy: 0.8301 - val_loss: 0.5080 - val_accuracy: 0.8211
Epoch 7/30
49/49 [==============================] - 2s 40ms/step - loss: 0.4631 - accuracy: 0.8496 - val_loss: 0.4604 - val_accuracy: 0.8396
Epoch 8/30
49

Now we will be evaluating the loss and accuracy of our model on testing data.

In [ ]:
loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 2s 2ms/step - loss: 0.2896 - accuracy: 0.8856


Now we are going to take a random review from our test dataset and check wether our model produces correct output or not

In [ ]:
index=np.random.randint(1,1000)
user_review=test_reviews.loc[index]
print(user_review)

Reviews      <START little <UNK> needs to fly was my first ...
Sentiment                                             positive
Name: 318, dtype: object


As we can see the sentiment for the above review is positive, now we are going to take the integer format of this particular review which we already have in our preprocessed test data and then give it as an input to our model to check the prediction of our model.

In [ ]:
user_review=test_data[index]
user_review=np.array([user_review])
if (model.predict(user_review)>0.5).astype("int32"):
  print("positive sentiment")
else:
  print("negative sentiment")

1/1 [==============================] - 0s 92ms/step
positive sentiment
